In [5]:
import pandas as pd
import numpy as np
import neptune
from neptune.utils import stringify_unsupported

from utils_neptune import get_latest_dataset
import os
NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='

PROJECT_ID = 'revivemed/Survival-RCC'


In [6]:
data_dir= '/app/finetune_data'
output_dir = '/app/mz_embed_engine/output'
local_dir = os.path.expanduser('~/saved_models')

In [ ]:
def get_pretrained_encoder(randinit=False,load_dir=None):
    if load_dir is None:
        load_dir = os.path.expanduser('~/PRETRAINED_MODEL')
        os.makedirs(load_dir,exist_ok=True)

    encoder_kwargs_path = os.path.join(load_dir,'encoder_kwargs.json')
    encoder_state_path =  os.path.join(load_dir,'encoder_state.pt')

    if (not os.path.exists(encoder_kwargs_path)) or (not os.path.exists(encoder_state_path)):

        neptune_model = neptune.init_model(project='revivemed/Survival-RCC',
            api_token= NEPTUNE_API_TOKEN,
            with_id='SUR-MOD',
            mode="read-only")

        if not os.path.exists(encoder_state_path):
            neptune_model['model/encoder_state'].download(encoder_state_path)
        if not os.path.exists(encoder_kwargs_path):
            encoder_kwargs = neptune_model['original_kwargs/encoder_kwargs'].fetch()
            if 'input_size' not in encoder_kwargs:
                encoder_kwargs['input_size'] = 2736
            if 'kind' not in encoder_kwargs:
                encoder_kwargs['kind'] = 





In [3]:
neptune_model = neptune.init_model(project='revivemed/Survival-RCC',
            api_token= NEPTUNE_API_TOKEN,
            with_id='SUR-MOD',
            mode="read-only")   

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/m/SUR-MOD


In [10]:
neptune_model.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/Survival-RCC/m/SUR-MOD/metadata


In [7]:
neptune_model['model/encoder_state'].download(f'{local_dir}/pretrained_encoder.pt')

Fetching file...: 0 [00:00, ?/s]

In [8]:
encoder_kwargs = neptune_model['original_kwargs/encoder_kwargs'].fetch()

In [9]:
encoder_kwargs

{'activation': 'leakyrelu',
 'dropout_rate': 0,
 'hidden_size_mult': 1.5,
 'latent_size': 108,
 'num_hidden_layers': 3,
 'use_batch_norm': False}

In [ ]:
    model_dir = f'{local_dir}/{run_id}/{task_id}'
    components_dir = f'{local_dir}/{run_id}/{task_id}/components'
    os.makedirs(components_dir, exist_ok=True)
    model_files = os.listdir(components_dir)
    
    if (len(model_files) < 4):

        run = neptune.init_run(project='revivemed/RCC',
            api_token= NEPTUNE_API_TOKEN,
            with_id=run_id,
            mode="read-only")   
        run_struc= run.get_structure()

        original_kwargs=run[task_id+'/original_kwargs'].fetch()
        json.dump(original_kwargs,open(f'{components_dir}/original_kwargs.json','w'),indent=4)
        substruc = get_sub_struc_from_path(run_struc,f'{task_id}/models')
        for key in substruc.keys():
            if 'info' in key:
                run[f'{task_id}/models/{key}'].download(f'{components_dir}/{key}.json')
            elif 'state' in key:
                run[f'{task_id}/models/{key}'].download(f'{components_dir}/{key}.pt')

        run.stop()
        model_files = os.listdir(components_dir)

In [ ]:
def run_model(model_name, params, data_dict, train_name='train', 
              eval_name='val', n_repeat=1,run_dict=None):
    
    if run_dict:
        run_dict['model_name'] = model_name
        run_dict['params'] = stringify_unsupported(params)
        run_dict['target_col'] = data_dict[train_name]['target_col']
        for set_name in data_dict.keys():
            if 'target_col':
                target_col = data_dict[set_name]['target_col']
                run_dict[f'dataset/{set_name}/target_col'] = target_col
            if 'X_file' in data_dict[set_name].keys():
                run_dict[f'dataset/{set_name}/X'].track_files(data_dict[set_name]['X_file'])
            if 'y_file' in data_dict[set_name].keys():
                run_dict[f'dataset/{set_name}/y'].track_files(data_dict[set_name]['y_file'])

    
    val_auc_list = []
    train_auc_list = []
    for _ in range(n_repeat):
        model = fit_model(model_name, params, data_dict, train_name)
        
        ## Classifier Specific Code
        train_acc, train_auc = evaluate_model(model, data_dict, train_name)
        val_acc, val_auc = evaluate_model(model, data_dict, eval_name)
        if run_dict:
            run_dict['metrics/train/accuracy'].append(train_acc)
            run_dict['metrics/train/auc'].append(train_auc)
            run_dict['metrics/val/accuracy'].append(val_acc)
            run_dict['metrics/val/auc'].append(val_auc)

        val_auc_list.append(val_auc)
        train_auc_list.append(train_auc)

    val_auc = np.mean(val_auc_list)
    train_auc = np.mean(train_auc_list)
    return val_auc, train_auc